In [17]:
def ReadFile(unformated_data,correct_misspelled):
    dummy=[]
    f=open("data.txt","r")
    for line in f:
        line=line.split(" ")
        #print(line)
        for word in line:
            unformated_data.append(word)
    
    f.close()
    
    f=open("misspellings.txt","r")
    counter=0
    for line in f:
        if counter==0:
            counter+=1
            continue
        
        line=line.split()
        #print(line)
        correct_misspelled.append(line)
    
    f.close()
    return unformated_data,correct_misspelled

In [18]:
unformated_data=[]
correct_misspelled=[]
unformated_data,correct_misspelled=ReadFile(unformated_data,correct_misspelled)

Unigrams={}

In [19]:
def GenerateUnigrams(Unigrams):
    for word in unformated_data:
        if word not in Unigrams:
            Unigrams[word]=1
        else:
            Unigrams[word]+=1
    
    corpus_length=len(unformated_data)
    for key in Unigrams:
        prob=Unigrams[key]
        prob/=corpus_length
        Unigrams[key]=prob
    
    return Unigrams
    
    
    

In [20]:
Unigrams=GenerateUnigrams(Unigrams)

In [21]:
def GenerateTable():
    Table={}
    Characters=['@','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    for character in Characters:
        for next_character in Characters:
            Table[(character,next_character)]=0
    
    return Table

In [22]:
Insert=GenerateTable()
delete=GenerateTable()
Substitute=GenerateTable()
Transpose=GenerateTable()

In [23]:
def Substitution(correct,mistyped):
    counter=0
    for i in range(len(correct)):
        if correct[i]!=mistyped[i]:
            counter+=1
            #print(correct[i],mistyped[i])
            Substitute[(correct[i],mistyped[i])]+=1
            return counter,correct[i],mistyped[i]

In [24]:
def Transposed(correct,mistyped):
    for i in range(len(correct)):
        for j in range(len(mistyped)):
            if correct[i]!=mistyped[j]:
                if j+1<len(mistyped):
                    if correct[i]==mistyped[j+1]:
                        Transpose[(correct[i],mistyped[j])]+=1
                        return correct[i-1],mistyped[i]

In [25]:
def Insertion(correct,mistyped):
    for i in range(len(mistyped)):
        if i==0:
            if correct[0]!=mistyped[0]:
                Insert[('@',mistyped[0])]+=1
                return '@',mistyped[0]
        
        try:
            if correct[i]!=mistyped[i]:
                Insert[(correct[i-1],mistyped[i])]+=1
                return correct[i-1],mistyped[i]
        
        except IndexError:
            Insert[(correct[i-1],mistyped[i])]+=1
            return correct[i-1],mistyped[i]
            
        
            
        
        

In [26]:
def Deletion(correct,mistyped):
    for i in range(len(correct)):
        if i==0:
            if correct[0]!=mistyped[0]:
                delete[('@',correct[0])]+=1
                return '@',correct[0]
                
        try:
            if correct[i]!=mistyped[i]:
                delete[(mistyped[i-1],correct[i])]+=1
                return mistyped[i-1],correct[i]
        
        except IndexError:
            delete[(mistyped[i-1],correct[i])]+=1
            return mistyped[i-1],correct[i]
            

In [27]:
def Probability(letter1,letter2,operation_matrix,correct_word,Unigrams):
    total_operations=sum(operation_matrix.values())
    probability=operation_matrix[(letter1,letter2)]/total_operations
    probability*=Unigrams[correct_word]
    return probability
    

In [28]:
def Suggested_Autocorrected(typo):
    autocorrected=""
    current=0
    previous=0
    counter=0
    word_candidate={}
    for i in range(len(correct_misspelled)):
        word=""
        #correct_misspelled.append(typo)
        next_range=len(correct_misspelled[i])
        for j in range(1,next_range):
            complete=correct_misspelled[i]
            complete.append(typo)
            correct=complete[0]
            length=len(correct)
            correct=correct[:length-1]
            if correct in Unigrams.keys():
                mistyped=complete[j]
                if mistyped!=typo: 
                    if len(correct)==len(mistyped):
                        counter,x,y=Substitution(correct,mistyped)
                        if counter==0:
                            x,y=Transposed(correct,mistyped)
                            current=Probability(x,y,Transpose,correct,Unigrams)
                            counter+=1

                        else:
                            current=Probability(x,y,Substitute,correct,Unigrams)
                            counter+=1

                    elif len(correct)>len(mistyped):
                        x,y=Deletion(correct,mistyped)
                        current=Probability(x,y,delete,correct,Unigrams)
                        counter+=1

                    elif len(correct)<len(mistyped):
                        x,y=Insertion(correct,mistyped)
                        current=Probability(x,y,Insert,correct,Unigrams)
                        counter+=1


                    if current>previous:
                        word=mistyped
                        previous=current
            
                else:
                    if len(correct)==len(mistyped):
                        counter,x,y=Substitution(correct,mistyped)
                        
                        if counter==0:
                            x,y=Transposed(correct,mistyped)
                            current=Probability(x,y,Transpose,correct,Unigrams)
                            counter+=1
                            

                        else:
                            current=Probability(x,y,Substitute,correct,Unigrams)
                            counter+=1
                            

                    elif len(correct)>len(mistyped):
                        x,y=Deletion(correct,mistyped)
                        current=Probability(x,y,delete,correct,Unigrams)
                        counter+=1
                        
                        
                    elif len(correct)<len(mistyped):
                        x,y=Insertion(correct,mistyped)
                        current=Probability(x,y,Insert,correct,Unigrams)
                        counter+=1
                        
                    
                        
                    
                    if current<previous:
                        
                        autocorrected=correct
                        return autocorrected
                    
                    else:
                    
                    
                        previous=0
                        current=0 

            else:
                previous=0
                break

        if counter!=0:
            word_candidate[word]=correct
            counter=0
            


In [33]:
typo=input("Enter a typo: ")
autocorrected=Suggested_Autocorrected(typo)
print("Autocorrected: ",autocorrected)


Enter a typo: wapasr
Autocorrected:  wapas
